In [ ]:
import json
from typing import Any, Dict
from matplotlib import pyplot
from matplotlib.axes._axes import Axes  # For typing purposes
import numpy as np
import pyperf
from scipy import stats

In [ ]:
def get_data(result: Dict[str, Any]) -> Dict[str, np.ndarray]:
    """Parse data"""
    results = {}

    for benchmark in result["benchmarks"]:
        if "metadata" in benchmark:
            name = benchmark["metadata"]["name"]
        else:
            name = result["metadata"]["name"]
        data = []
        for run in benchmark["runs"]:
            data.extend(run.get("values", []))
        results[name] = np.array(data, dtype=np.float64)
        results[name].sort()

    return results

In [ ]:
# Load data
run = "run1"
file1 = f"results/{run}/bm-python3-11-3-high-01.json"
file2 = f"results/{run}/bm-python3-11-3-high-02.json"

with open(file1) as fb, open(file2) as fh:
    base = get_data(json.load(fb))
    head = get_data(json.load(fh))

In [ ]:
# Make sure that benchmarks match
if base.keys() != head.keys():
    raise Exception("The benchmarking suites are not the same size")

In [ ]:
# Clean data
remove = []

for name in base.keys():
    sig, _ = pyperf._utils.is_significant(base[name], head[name])

    # if not sig:
    #     remove.append(name)
    # else:
        # Remove outliers
    base[name] = base[name][:45]
    head[name] = head[name][:45]

# for name in remove:
#     del head[name]
#     del head[name]

In [ ]:
def generate_table(ax: Axes, base: np.ndarray, head: np.ndarray) -> None:
    c_labels = ["Base", "Head"]
    r_labels = [
        "Mean", 
        "Variance", 
        "Skewness", 
        "Variation", 
        "Minimum", 
        "Maximum",
    ]

    data = [
        [base.mean(), head.mean()],  # Empirical mean
        [base.var(), head.var()],  # Variance
        [stats.skew(base), stats.skew(head)],  # Sample Skewness
        [stats.variation(base), stats.variation(head)],  # Coefficient of variation
        [base.min(), head.min()],  # Minimum
        [base.max(), head.max()],  # Maximum
    ]
    data = [[round(b, 4), round(h, 4)] for b, h in data]  # Round to 4 decimal points

    table = ax.table(cellText=data, loc='center', cellLoc='center', rowLabels=r_labels, colLabels=c_labels)
    table.set_fontsize(11)

    _, p_val = stats.ttest_ind(base, head)  # P-value
    p_val = round(p_val, 4)

    # I think matplotlib enforces the same number of cells in a row.
    # Therefore i use two cells and remove one border.
    table.add_cell(7, 0, text=p_val, loc='center', width=0.5, height=0.0981241).visible_edges = 'BTL'
    table.add_cell(7, 1, text='', width=0.5, height=0.0981241).visible_edges = 'BRT'
    table.add_cell(7, -1, text='P-value', width=0.5, height=0.0981241, loc='left')

    
    ax.axis("off")

In [ ]:
def generate_dist_plot(ax: Axes, name: str, base: np.ndarray, head: np.ndarray) -> None:
    # The density set to `True` makes the integral of the histogram 1.
    ax.hist(base, alpha=0.5, label='Base', density=True)
    ax.hist(head, alpha=0.5, label='Head', density=True)

    x1 = np.linspace(base.mean() - 3 * base.std(), base.mean() + 3 * base.std(), 100)
    y1 = stats.norm.pdf(x1, base.mean(), base.std())
    ax.plot(x1, y1, label='Base (Normal Dist.)', color="blue")

    x2 = np.linspace(head.mean() - 3 * head.std(), head.mean() + 3 * head.std(), 100)
    y2 = stats.norm.pdf(x2, head.mean(), head.std())
    ax.plot(x2, y2, label='Head (Normal Dist.)', color="orange")

    ax.set_ylabel(name)
    ax.legend()


In [ ]:
# _, axs = pyplot.subplots(3, 2, figsize=(10, 2.5 * 3), layout="constrained", width_ratios=[.7, .3])
_, axs = pyplot.subplots(len(base), 2, figsize=(10, 2.5 * len(base)), layout="constrained", width_ratios=[.7, .3])

ax_counter = 0
for name in base.keys():
    generate_dist_plot(axs[ax_counter][0], name, base[name], head[name])
    
    generate_table(axs[ax_counter][1], base[name], head[name])
    ax_counter += 1

    # if ax_counter == 3:
    #     break

pyplot.show()
